### Bibliotecas

In [ ]:
"""
%pip install aeon
%pip install tsfresh
%pip install tslearn
%pip install tensorflow
%pip install keras
%pip install pywavelets

"""

In [ ]:
import pandas as pd
import numpy as np

from aeon.datasets import load_classification
from aeon.datasets.tsc_data_lists import univariate_equal_length
from aeon.classification.interval_based import SupervisedTimeSeriesForest, TimeSeriesForestClassifier

#from tsfresh import extract_features, select_features
#from tsfresh.feature_extraction import MinimalFCParameters

from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.piecewise import PiecewiseAggregateApproximation, SymbolicAggregateApproximation

import pywt
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from scipy.fftpack import fft
from numba import jit
import timeit
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

### Loading data

In [ ]:
def load_data(dataset):
    # LabelEncoder para labels alvo
    le = LabelEncoder()

    # Carregar conjunto de dados do repositório UCR
    X_train, y_train = load_classification(dataset, split="TRAIN")
    X_test, y_test = load_classification(dataset, split="test")

    # Formatar o conjunto de dados para 2D
    features_train = X_train.reshape(X_train.shape[0], -1)
    features_test = X_test.reshape(X_test.shape[0], -1)

    # Ajustar e transformar as labels alvo
    target_train = le.fit_transform(y_train)
    target_test = le.transform(y_test)

    return features_train, features_test, target_train, target_test


### Function transform data

In [ ]:
def choose_wavelet(X):
    min_variance = float('inf')
    best_wavelet = None
    candidate_wavelets = ['db1', 'db2', 'db3', 'db4', 'db5', 'db6', 'db7', 'db8', 'db9']

    for wavelet_type in candidate_wavelets:
        _, coeffs_cD = pywt.dwt(X, wavelet_type, axis=1)
        total_variance = np.var(coeffs_cD)

        if total_variance < min_variance:
            min_variance = total_variance
            best_wavelet = wavelet_type
    return str(best_wavelet)

def transform_data_math(X, wavelet):
    n_sax_symbols = int(X.shape[1] / 4)
    n_paa_segments = int(X.shape[1] / 4)

    X_fft = np.abs(fft(X, axis=1))

    coeffs_cA, coeffs_cD = pywt.dwt(X, wavelet=wavelet, axis=1, mode='constant')
    X_dwt = np.hstack((coeffs_cA, coeffs_cD))

    paa = PiecewiseAggregateApproximation(n_segments=n_paa_segments)
    X_paa_ = paa.inverse_transform(paa.fit_transform(X))
    X_paa = X_paa_.reshape(X_paa_.shape[0], -1)
    df_PAA = pd.DataFrame(X_paa)
    #df_PAA['id'] = range(len(df_PAA))
    
    sax = SymbolicAggregateApproximation(n_segments=n_paa_segments, alphabet_size_avg=n_sax_symbols)
    X_sax_ = sax.inverse_transform(sax.fit_transform(X))
    X_sax = X_sax_.reshape(X_sax_.shape[0], -1)
    df_SAX = pd.DataFrame(X_sax)
    #df_SAX['id'] = range(len(df_SAX))

    data_X = TimeSeriesScalerMeanVariance().fit_transform(X)
    data_X.resize(data_X.shape[0], data_X.shape[1])
    df_X = pd.DataFrame(data_X)
    #df_X['id'] = range(len(df_X))

    data_FFT = TimeSeriesScalerMeanVariance().fit_transform(X_fft)
    data_FFT.resize(data_FFT.shape[0], data_FFT.shape[1])
    df_FFT = pd.DataFrame(data_FFT)
    #df_FFT['id'] = range(len(df_FFT))

    data_DWT = TimeSeriesScalerMeanVariance().fit_transform(X_dwt)
    data_DWT.resize(data_DWT.shape[0], data_DWT.shape[1])
    df_DWT = pd.DataFrame(data_DWT)
    #df_DWT['id'] = range(len(df_DWT))
    
    return {'X': df_X, 'FFT': df_FFT, 'DWT': df_DWT, 'PAA': df_PAA, 'SAX': df_SAX}

def extract_features_from_data(data):
    extracted_features_list = []
    for name, df in data.items():
        features = extract_features(df, default_fc_parameters=MinimalFCParameters(), disable_progressbar=True, column_id='id')
        extracted_features_list.append(features)
    return extracted_features_list

def select_best_features(extracted_features_dict):
    best_features_list = []
    for name, features in extracted_features_dict.items():
        best_features = select_features(features)
        best_features['Method'] = name
        best_features_list.append(best_features)
    concatenated_df = pd.concat(best_features_list, axis=1)
    return concatenated_df.to_numpy() 

In [ ]:
    transformed_X = np.stack([df_X, df_FFT, df_DWT, df_PAA, df_SAX], axis=-1)
    
    return transformed_X

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate

# Crie uma entrada para cada representação de dados
input_X = Input(shape=(1, 128, 1))
input_FFT = Input(shape=(1, 128, 1))
input_DWT = Input(shape=(1, 128, 1))
input_PAA = Input(shape=(1, 128, 1))
input_SAX = Input(shape=(1, 128, 1))

inputs = [input_X, input_FFT, input_DWT, input_PAA, input_SAX]

# Crie uma sub-rede para cada entrada
subnets = []
for inp in inputs:
    x = Conv2D(32, (3, 3), activation='relu')(inp)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    subnets.append(x)

# Concatene as saídas das sub-redes
x = concatenate(subnets)

# Adicione algumas camadas densas
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# Camada de saída
output = Dense(1, activation='sigmoid')(x)

# Crie o modelo
model = Model(inputs=inputs, outputs=output)

# Compile o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
import torch
from torch import nn, optim
import learn2learn as l2l

# Defina a arquitetura da sua rede neural
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Crie uma instância do modelo e um otimizador
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Crie uma instância do algoritmo MAML
maml = l2l.algorithms.MAML(model, lr=1e-3, first_order=False)

# Suponha que temos um dataloader que carrega as tarefas
for iteration, task in enumerate(dataloader):
    learner = maml.clone()  # Crie um novo learner (i.e., uma nova instância do modelo)
    
    # "task" é uma tarefa, ou seja, um conjunto de dados de treinamento
    X_train, y_train = task

    # Adapte o modelo à tarefa
    learner.adapt(torch.optim.Adam, X_train, y_train, n_steps=5)

    # Calcule a perda no conjunto de dados de treinamento
    train_preds = learner(X_train)
    train_loss = loss(train_preds, y_train)

    # Atualize os pesos do modelo
    train_loss.backward()
    optimizer.step()
    optimizer.zero_grad()


### AmazonForestClassifier

In [ ]:
class CombinedDecisionForest:
    def __init__(self):
        self.clf1 = RandomForestClassifier()
        self.clf2 = ExtraTreesClassifier()
        self.clf3 = SupervisedTimeSeriesForest()
        self.clf4 = TimeSeriesForestClassifier()
        self.classifiers = [self.clf1, self.clf2, self.clf3, self.clf4]
        self.clf_weights = None

    def fit(self, X, y):
        for clf in self.classifiers:
            clf.fit(X, y)

        train_preds = [clf.predict(X) for clf in self.classifiers]
        accuracies = [accuracy_score(y, preds) for preds in train_preds]

        self.clf_weights = np.array(accuracies)
        self.clf_weights /= np.sum(self.clf_weights)

    def predict_proba(self, X):
        probs = [clf.predict_proba(X) for clf in self.classifiers]
        combined_probs = np.sum([prob * weight for prob, weight in zip(probs, self.clf_weights)], axis=0)
        return combined_probs / np.sum(combined_probs, axis=1, keepdims=True)

    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)

In [ ]:
"""class CombinedDecisionForest:
    def __init__(self):
        self.clf1 = RandomForestClassifier()
        self.clf2 = ExtraTreesClassifier()
        self.clf3 = SupervisedTimeSeriesForest()
        self.clf4 = TimeSeriesForestClassifier()
        self.clf_weights = None

    def fit(self, X, y):
        classifiers = [self.clf1, self.clf2, self.clf3, self.clf4]  # Lista de classificadores

        for clf in classifiers:
            clf.fit(X, y) # Calculando os pesos com base na acurácia
        train_preds = [clf.predict(X) for clf in classifiers]
        accuracies = [accuracy_score(y, preds) for preds in train_preds]

        self.clf_weights = np.array(accuracies) ** 4
        self.clf_weights /= np.sum(self.clf_weights) # Normalização dos pesos

    def predict_proba(self, X):
        classifiers = [self.clf1, self.clf2, self.clf3, self.clf4]  # Lista de classificadores

        probs = [clf.predict_proba(X) for clf in classifiers]
        combined_probs = np.sum([prob * weight for prob, weight in zip(probs, self.clf_weights)], axis=0)

        return combined_probs / np.sum(combined_probs, axis=1, keepdims=True)

    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)"""

### Train/Predict

In [ ]:
def train_classifier(X_train, y_train, wavelet=None):
    trained_models = {}  # Salvar modelos treinados para cada transformação
    X_train_transformed = transform_data_math(X_train, wavelet=wavelet)  # Transformar todo o conjunto de treino
    # Treinar um modelo para cada transformação e salvar no dicionário
    for rep, X_trans in X_train_transformed.items():
        model = SupervisedTimeSeriesForest()
        model.fit(X_trans, y_train)
        trained_models[rep] = model
        
    # Preparar dados para o meta-classificador
    meta_features = []
    for i in range(X_train.shape[0]):
        instance_features = []
        for rep, model in trained_models.items():
            proba = model.predict_proba(np.array(X_train_transformed[rep].iloc[i,:]).reshape(1, -1))
            instance_features.extend(proba.flatten())
        meta_features.append(instance_features)
    
    meta_features = np.array(meta_features)
    
    meta_classifier = CombinedDecisionForest()
    meta_classifier.fit(meta_features, y_train)
    
    return trained_models, meta_classifier

def predict_classifier(X_test, trained_base_model, trained_meta_classifier, wavelet=None):
    predictions = []
    meta_features_test = []

    for i in tqdm(range(len(X_test)), ascii=True, colour='green', desc="Testing"):
        x_instance = X_test[i].reshape(1, -1)
        x_transformed = transform_data_math(x_instance, wavelet=wavelet)

        instance_features = []
        for rep, X_trans in x_transformed.items():
            proba = trained_base_model[rep].predict_proba(X_trans.values.reshape(1, -1))
            instance_features.extend(proba.flatten())

        meta_feature = np.array(instance_features).reshape(1, -1)
        predictions.append(trained_meta_classifier.predict(meta_feature)[0])

        meta_features_test.append(meta_feature.flatten())

    meta_features_test = np.array(meta_features_test)

    return predictions


### Validando o modelo

In [ ]:
dataset_quali_list =  ['ElectricDevices']
dataset_full_list = ['PhalangesOutlinesCorrect','CricketZ','MiddlePhalanxOutlineAgeGroup','ECG5000','ShapeletSim','MiddlePhalanxTW','Symbols','EOGHorizontalSignal','Ham','UMD','HouseTwenty','MiddlePhalanxOutlineCorrect','Wafer','DistalPhalanxTW','CricketY','FacesUCR','FiftyWords','Mallat','Strawberry','ProximalPhalanxOutlineAgeGroup','MixedShapesRegularTrain','SmallKitchenAppliances','GunPointOldVersusYoung','Wine','ProximalPhalanxOutlineCorrect','WordSynonyms', 'RefrigerationDevices','Yoga','CinCECGTorso','ChlorineConcentration','ToeSegmentation1','TwoLeadECG','ProximalPhalanxTW','InsectEPGSmallTrain','WormsTwoClass','PowerCons','InsectEPGRegularTrain','GunPointMaleVersusFemale','DistalPhalanxOutlineCorrect','ItalyPowerDemand','InsectWingbeatSound','NonInvasiveFetalECGThorax2','CricketX','Haptics','EOGVerticalSignal','MixedShapesSmallTrain','Meat','SemgHandGenderCh2','ToeSegmentation2','NonInvasiveFetalECGThorax1','FreezerSmallTrain','OSULeaf','BirdChicken','HandOutlines','BeetleFly','ACSF1','DistalPhalanxOutlineAgeGroup','FreezerRegularTrain']
problematicos = ['Crop','EthanolLevel','ElectricDevices','FordB','ShapesAll','StarLightCurves','Phoneme', 'Computers','InlineSkate','PigAirwayPressure', 'PigCVP','FordA','MedicalImages','PigArtPressure', 'UWaveGestureLibraryX','UWaveGestureLibraryY', 'UWaveGestureLibraryZ', 'UWaveGestureLibraryAll', 'TwoPatterns']

In [ ]:
runned = {'Adiac',
'Beef',
'BeetleFly',
'BME',
'Car',
'CBF',
'Coffee',
'Computers',
'Chinatown',
'ChlorineConcentration',
'DistalPhalanxOutlineCorrect',
'DistalPhalanxTW',
'ECG200',
'ECG5000',
'ECGFiveDays',
'EOGHorizontalSignal',
'Earthquakes',
'EthanolLevel',
'FaceFour',
'FacesUCR',
'GunPoint',
'GunPointMaleVersusFemale',
'HandOutlines',
'Lightning2',
'Lightning7',
'MedicalImages',
'MiddlePhalanxOutlineAgeGroup',
'MoteStrain',
'NonInvasiveFetalECGThorax1',
'NonInvasiveFetalECGThorax2',
'OliveOil',
'PigAirwayPressure',
'PowerCons',
'SonyAIBORobotSurface1',
'SonyAIBORobotSurface2',
'ShapeletSim',
'ShapesAll',
'SmoothSubspace',
'SyntheticControl',
'Trace',
'TwoPatterns',
'SemgHandMovementCh2',
'PigArtPressure',
'Symbols',
'ItalyPowerDemand',
'PhalangesOutlinesCorrect',
'LargeKitchenAppliances',
'SemHandGenderCh2'
'DiatomSizeReduction',
'PigCVP',
'FordB',
'SemgHandSubjectCh2',
'ScreenType',  
'Yoga',  
'UWaveGestureLibraryY',
'SemgHandGenderCh2',  
'ElectricDevices',
'OSULeaf',
'GunPointAgeSpan',
'ToeSegmentation1',
'ToeSegmentation2',
'CricketZ',
'Strawberry',
'CricketY',
'WordSynonyms',
'RefrigerationDevices',  
'MixedShapesSmallTrain',  
'Wafer',
'Worms',  
'Phoneme',
'GunPointOldVersusYoung',
'UMD',  
'InsectWingbeatSound',
'TwoLeadECG',
'Rock',
'DistalPhalanxOutlineAgeGroup',
'ACSF1',
'FiftyWords',
'HouseTwenty',
'Meat',
'FreezerRegularTrain',
'Crop',
'CricketX',
'UWaveGestureLibraryX',
'ProximalPhalanxTW',
'SwedishLeaf',
'FreezerSmallTrain',
'ArrowHead',
'Fish',
'StarLightCurves',
'InsectEPGSmallTrain',
'WormsTwoClass',
'CinCECGTorso',
'MiddlePhalanxOutlineCorrect',
'Haptics',
'Ham',
'SmallKitchenAppliances',
'UWaveGestureLibraryAll',
'ProximalPhalanxOutlineCorrect',
'InsectEPGRegularTrain',
'ProximalPhalanxOutlineAgeGroup'}

In [ ]:
selected_data = univariate_equal_length - runned

In [ ]:
accuracy_data = []
for dataset_name in selected_data:
    features_train, features_test, target_train, target_test = load_data(dataset_name)
    best_wavelet = choose_wavelet(features_train)
    
    trained, clf = train_classifier(features_train, target_train, best_wavelet)
    predictions = predict_classifier(features_test, trained, clf, best_wavelet)

    accuracy = accuracy_score(target_test, predictions)
        
    accuracy_data.append({'Dataset Name': dataset_name, 'Accuracy': accuracy})
    
    print(f"Acurácia {dataset_name}: {accuracy}")
    
accuracy_df = pd.DataFrame(accuracy_data)


In [ ]:
accuracy_df
accuracy_df.to_csv('ModelAM_STSF71.csv')

### DynamicAmazonClassifier

In [ ]:
from sklearn.linear_model import RidgeClassifierCV

accuracy_data = []
for dataset_name in univariate_equal_length:
    # Carregue os dados de treinamento e teste
    features_train, features_test, target_train, target_test = load_data(dataset_name)
    best_wavelet = choose_wavelet(features_train)
    
    feature_extractor = CombinedDecisionForest()
    feature_extractor.fit(features_train, target_train)
    
    train_features = feature_extractor.predict_proba(features_train)
    test_features = feature_extractor.predict_proba(features_test)
    
    meta_model = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
    
    meta_model.fit(train_features, target_train)
    
    predictions = meta_model.predict(test_features)
    
    accuracy = accuracy_score(target_test, predictions)
        
    accuracy_data.append({'Dataset Name': dataset_name, 'Accuracy': accuracy})
    
    print(f"Accuracy {dataset_name}: {accuracy}")
    
accuracy_df = pd.DataFrame(accuracy_data)

In [ ]:
accuracy_df

In [ ]:
accuracy_df.to_csv('ModelAMM_FeatureWorking.csv')

### Representação temporal de uma transformada rápida de Fourier

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Criação dos dados
t = np.linspace(0, 1, 250, endpoint=False)
a = 0.02
f = 5
x = 0.9 * np.sin(2 * np.pi * 3 * f * t + .1)
x += a * np.cos(2 * np.pi * 312 * f * t + .1)
x += a * np.cos(2 * np.pi * 2000 * f * t)
x += a * np.sin(2 * np.pi * 5000 * f * t)
x += 0.8 * np.cos(2 * np.pi * 2.3e4 * f * t)
x += 0.3 * np.cos(2 * np.pi * 1.3e5 * f * t + .1)
x += 0.3 * np.cos(2 * np.pi * 2.3e5 * f * t + .1)
x += a * np.cos(2 * np.pi * 2e6 * f * t)

# Transformada de Fourier
y = np.fft.fft(x)
freq = np.fft.fftfreq(len(x), d=t[1] - t[0])

# Plotagem do gráfico
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.plot(t, x)
plt.title('Série Temporal')

plt.subplot(122)
plt.plot(freq, np.abs(y))
plt.title('Espectro de Frequência')

plt.tight_layout()
plt.show()
